In [1]:
%pip install pandas -U -q
%pip install numpy -U -q
%pip install yfinance -U -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List, Dict

import pandas as pd
import numpy as np
import yfinance as yf

## Unir as ações que compõem os índices IBOV e IDIV

In [3]:
def get_tickers_from_market_index(source: str):
  market_index = pd.read_csv(source)
  market_index.drop(columns="Part. (%)", inplace=True)
  market_index.dropna(inplace=True)
  return market_index.iloc[:, 0].to_list()

def merge_tickers_from_market_indexes(*args: List[str]) -> List[str]:
  result = set()

  for market_index in args:
    result = result.union(market_index)

  return list(result)

In [4]:
ibov_composition = get_tickers_from_market_index("./assets/IBOV-composition.csv")
ibov_composition

['RRRP3',
 'ALSO3',
 'ALPA4',
 'ABEV3',
 'ARZZ3',
 'ASAI3',
 'AZUL4',
 'B3SA3',
 'BBSE3',
 'BBDC3',
 'BBDC4',
 'BRAP4',
 'BBAS3',
 'BRKM5',
 'BRFS3',
 'BPAC11',
 'CRFB3',
 'BHIA3',
 'CCRO3',
 'CMIG4',
 'CIEL3',
 'COGN3',
 'CPLE6',
 'CSAN3',
 'CPFE3',
 'CMIN3',
 'CVCB3',
 'CYRE3',
 'DXCO3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENGI11',
 'ENEV3',
 'EGIE3',
 'EQTL3',
 'EZTC3',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GOLL4',
 'NTCO3',
 'SOMA3',
 'HAPV3',
 'HYPE3',
 'IGTI11',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'RENT3',
 'LWSA3',
 'LREN3',
 'MGLU3',
 'MRFG3',
 'BEEF3',
 'MRVE3',
 'MULT3',
 'PCAR3',
 'PETR3',
 'PETR4',
 'RECV3',
 'PRIO3',
 'PETZ3',
 'RADL3',
 'RAIZ4',
 'RDOR3',
 'RAIL3',
 'SBSP3',
 'SANB11',
 'SMTO3',
 'CSNA3',
 'SLCE3',
 'SUZB3',
 'TAEE11',
 'VIVT3',
 'TIMS3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VAMO3',
 'VBBR3',
 'WEGE3',
 'YDUQ3']

In [5]:
idiv_composition = get_tickers_from_market_index("./assets/IDIV-composition.csv")
idiv_composition

['ABCB4',
 'ALUP11',
 'AURE3',
 'B3SA3',
 'BRSR6',
 'BBSE3',
 'BRAP4',
 'BBAS3',
 'AGRO3',
 'CXSE3',
 'CMIG3',
 'CMIG4',
 'CSMG3',
 'CPLE3',
 'CPLE6',
 'CPFE3',
 'CMIN3',
 'DIRR3',
 'EGIE3',
 'FESA4',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GRND3',
 'RANI3',
 'ITSA4',
 'JBSS3',
 'JHSF3',
 'KEPL3',
 'KLBN11',
 'LAVV3',
 'MRFG3',
 'LEVE3',
 'BEEF3',
 'PETR3',
 'PETR4',
 'PSSA3',
 'RAPT4',
 'ROMI3',
 'SANB11',
 'CSNA3',
 'TAEE11',
 'TASA4',
 'TGMA3',
 'VIVT3',
 'TRPL4',
 'TRIS3',
 'UNIP6',
 'USIM5',
 'VALE3',
 'WIZC3']

## Base de dados com com todos os atributos que compõem o fluxo de caixa

In [6]:
tickers = merge_tickers_from_market_indexes(ibov_composition, idiv_composition)

In [7]:
def format_tickers_to_yfinance(tickers: List[str]) -> List[str]:
  return [f"{ticker}.SA" for ticker in tickers]

tickers = format_tickers_to_yfinance(tickers)

In [22]:
cash_flow_data = pd.DataFrame()

for ticker in tickers:
    cashflow = yf.Ticker(ticker).quarterly_cashflow
    
    # Define o ticker como a coluna principal no MultiIndex
    cashflow.columns = pd.MultiIndex.from_product([[ticker], cashflow.columns])
    
    if cash_flow_data.empty:
        cash_flow_data = cashflow
        continue

    cash_flow_data = pd.concat([cash_flow_data, cashflow], axis=1)


In [23]:
cash_flow_data.dropna(inplace=True)
cash_flow_data

CSAN3.SA                                \
                            2023-06-30     2023-03-31     2022-12-31   
Free Cash Flow            1709467000.0    308884000.0   1145098000.0   
End Cash Position        15739290000.0  11385426000.0  13301716000.0   
Beginning Cash Position  11385426000.0  13301716000.0  15487103000.0   
Changes In Cash           4433235000.0  -1896770000.0  -2175234000.0   
Financing Cash Flow       2695912000.0  -2301659000.0   5534252000.0   

                                            CIEL3.SA                \
                            2022-09-30    2023-06-30    2023-03-31   
Free Cash Flow            2563249000.0 -1731199000.0  1131122000.0   
End Cash Position        15487103000.0   951896000.0  2351092000.0   
Beginning Cash Position  12679430000.0  2351092000.0  1882866000.0   
Changes In Cash           2907373000.0 -1399195000.0   468226000.0   
Financing Cash Flow       5106019000.0   332004000.0  -662896000.0   

                                                        EZTC3.SA              \
                           2022-12-31    2022-09-30   2023-06-30  2023-03-31   
Free Cash Flow           1436880000.0  -117603000.0 -154068000.0 -31196000.0   
End Cash Position        1882866000.0  1014558000.0   42473000.0  33602000.0   
Beginning Cash Position  1014558000.0  1756224000.0   33602000.0  41889000.0   
Changes In Cash           868308000.0  -741666000.0    1656000.0  -8287000.0   
Financing Cash Flow      -568572000.0  -782764000.0   69204000.0  38677000.0   

                         ...     ALPA4.SA                   B3SA3.SA  \
                         ...   2022-12-31   2022-09-30    2023-06-30   
Free Cash Flow           ... -431243000.0 -141413000.0  2075448000.0   
End Cash Position        ...  647517000.0  240984000.0   352574000.0   
Beginning Cash Position  ...  240984000.0  273099000.0   324523000.0   
Changes In Cash          ...  407010000.0  -30181000.0    31541000.0   
Financing Cash Flow      ...  804951000.0  157054000.0 -1453471000.0   

                                                                   \
                           2023-03-31    2022-12-31    2022-09-30   
Free Cash Flow           1309213000.0  1724533000.0  2426082000.0   
End Cash Position         324523000.0   362242000.0   318487000.0   
Beginning Cash Position   362242000.0   318487000.0   281502000.0   
Changes In Cash           -36359000.0    43755000.0    36985000.0   
Financing Cash Flow     -1309368000.0 -1694528000.0 -2396088000.0   

                            KEPL3.SA                                         
                          2023-06-30   2023-03-31   2022-12-31   2022-09-30  
Free Cash Flow            54843000.0   24528000.0   93833000.0   78805000.0  
End Cash Position        251482000.0  199128000.0  254454000.0  200402000.0  
Beginning Cash Position  199128000.0  254454000.0  200402000.0  208545000.0  
Changes In Cash           52354000.0  -55326000.0   54052000.0   -8143000.0  
Financing Cash Flow     -104949000.0    -995000.0  -51757000.0  -51690000.0  

[5 rows x 432 columns]

In [24]:
cash_flow_data.to_csv("./assets/brazilian_companies_complete_cashflow.csv")

## Base de dados  com os atributos mais relevantes para o cálculo do fluxo de caixa livre

O critério de escolha dos atributos mais relevantes é feito da seguinte maneira:

1. Cálculo da correlação de todos os atributos em relação ao atributo "Free Cash Flow" para cada ativo da base de dados;
2. Obter todos os atributos classificados como correlação forte ($cor \geq 0.7$) de cada ativo;
3. Obter os 3 atributos com maior frequência com base na etapa anterior.

Considerações importantes:

- Não foi escolhido o critério da correlação média dos atributos em relação ao "Free Cash Flow", pois pelo fato da média não ser uma medida resistente (sensível a outliers). Dessa forma, isso pode afetar o resultado de algoritmos de previsão do "Free Cash Flow", uma vez que pode ocorrer de um dos atributos selecionados como "mais relevantes" não explique a variação do "Free Cash Flow" de forma apropriada.

In [25]:
tickers = list(set([ticker for ticker, _ in cash_flow_data.columns]))
tickers

['CSAN3.SA',
 'CIEL3.SA',
 'EZTC3.SA',
 'SANB11.SA',
 'CYRE3.SA',
 'RRRP3.SA',
 'BRFS3.SA',
 'AGRO3.SA',
 'ALSO3.SA',
 'BHIA3.SA',
 'IRBR3.SA',
 'ITSA4.SA',
 'SUZB3.SA',
 'CMIG3.SA',
 'LREN3.SA',
 'VBBR3.SA',
 'BRKM5.SA',
 'TRIS3.SA',
 'RECV3.SA',
 'PSSA3.SA',
 'LWSA3.SA',
 'GOLL4.SA',
 'CRFB3.SA',
 'DXCO3.SA',
 'RAIL3.SA',
 'BBSE3.SA',
 'PETR3.SA',
 'DIRR3.SA',
 'EGIE3.SA',
 'ALUP11.SA',
 'MULT3.SA',
 'FESA4.SA',
 'KLBN11.SA',
 'ENEV3.SA',
 'RDOR3.SA',
 'EMBR3.SA',
 'JHSF3.SA',
 'BEEF3.SA',
 'FLRY3.SA',
 'BRAP4.SA',
 'LEVE3.SA',
 'VIVT3.SA',
 'TGMA3.SA',
 'JBSS3.SA',
 'BRSR6.SA',
 'ARZZ3.SA',
 'NTCO3.SA',
 'ELET6.SA',
 'PETR4.SA',
 'PCAR3.SA',
 'CSMG3.SA',
 'ABEV3.SA',
 'AURE3.SA',
 'PRIO3.SA',
 'VAMO3.SA',
 'WEGE3.SA',
 'GOAU4.SA',
 'TASA4.SA',
 'SLCE3.SA',
 'HYPE3.SA',
 'RENT3.SA',
 'COGN3.SA',
 'ROMI3.SA',
 'CCRO3.SA',
 'ASAI3.SA',
 'CXSE3.SA',
 'ITUB4.SA',
 'TIMS3.SA',
 'IGTI11.SA',
 'YDUQ3.SA',
 'EQTL3.SA',
 'RANI3.SA',
 'TOTS3.SA',
 'TRPL4.SA',
 'BBAS3.SA',
 'MGLU3.SA',
 'TAEE11

In [26]:
correlation_results = []

for ticker in tickers:
  df = cash_flow_data[ticker].transpose()
  correlation_matrix = df.corr()["Free Cash Flow"]

  # Filtra os atributos com a correlação média >= 0.7 em relação ao atributo "Free Cash Flow"
  high_corr = correlation_matrix[correlation_matrix >= 0.7].iloc[1:]

  # Atributos com correlação forte com a coluna "Free Cash Flow"
  most_relevant = high_corr.sort_values().index.to_list()
  correlation_results.append(most_relevant)


In [27]:
import itertools
import collections


def frequency_counter(matrix: List[List[any]], n: int | None = None) -> Dict[any, int]:
    # flatten the list using chain()
    flat_list = list(itertools.chain.from_iterable(matrix))

    # count the frequencies using Counter()
    frequencies = collections.Counter(flat_list)

    if not n:
        return frequencies
    
    # Get the N most frequent attributes
    frequencies = {
        attr: frequency
        for rank, (attr, frequency) in enumerate(sorted(frequencies.items(), key=lambda item: item[1], reverse=True))
        if rank < n
    }

    return frequencies

In [28]:
frequency_counter(correlation_results)

Counter({'Changes In Cash': 48,
         'End Cash Position': 33,
         'Financing Cash Flow': 18,
         'Beginning Cash Position': 3})

In [30]:
most_relevant = frequency_counter(correlation_results, 3)
most_relevant

{'Changes In Cash': 48, 'End Cash Position': 33, 'Financing Cash Flow': 18}

In [31]:
filter = cash_flow_data.index.isin(most_relevant)
cash_flow_data = cash_flow_data[filter]
cash_flow_data

CSAN3.SA                                \
                        2023-06-30     2023-03-31     2022-12-31   
End Cash Position    15739290000.0  11385426000.0  13301716000.0   
Changes In Cash       4433235000.0  -1896770000.0  -2175234000.0   
Financing Cash Flow   2695912000.0  -2301659000.0   5534252000.0   

                                        CIEL3.SA                              \
                        2022-09-30    2023-06-30    2023-03-31    2022-12-31   
End Cash Position    15487103000.0   951896000.0  2351092000.0  1882866000.0   
Changes In Cash       2907373000.0 -1399195000.0   468226000.0   868308000.0   
Financing Cash Flow   5106019000.0   332004000.0  -662896000.0  -568572000.0   

                                     EZTC3.SA              ...     ALPA4.SA  \
                       2022-09-30  2023-06-30  2023-03-31  ...   2022-12-31   
End Cash Position    1014558000.0  42473000.0  33602000.0  ...  647517000.0   
Changes In Cash      -741666000.0   1656000.0  -8287000.0  ...  407010000.0   
Financing Cash Flow  -782764000.0  69204000.0  38677000.0  ...  804951000.0   

                                      B3SA3.SA                              \
                      2022-09-30    2023-06-30    2023-03-31    2022-12-31   
End Cash Position    240984000.0   352574000.0   324523000.0   362242000.0   
Changes In Cash      -30181000.0    31541000.0   -36359000.0    43755000.0   
Financing Cash Flow  157054000.0 -1453471000.0 -1309368000.0 -1694528000.0   

                                      KEPL3.SA                            \
                       2022-09-30   2023-06-30   2023-03-31   2022-12-31   
End Cash Position     318487000.0  251482000.0  199128000.0  254454000.0   
Changes In Cash        36985000.0   52354000.0  -55326000.0   54052000.0   
Financing Cash Flow -2396088000.0 -104949000.0    -995000.0  -51757000.0   

                                  
                      2022-09-30  
End Cash Position    200402000.0  
Changes In Cash       -8143000.0  
Financing Cash Flow  -51690000.0  

[3 rows x 432 columns]

In [32]:
cash_flow_data.to_csv("./assets/brazilian_companies_most_relevant_attr_for_free_cash_flow.csv")